<a href="https://colab.research.google.com/github/kridtapon/Signal-Squeeze-Breakout/blob/main/Signal_Squeeze_Breakout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.5 MB/s eta 0:00:00


In [2]:
pip install --upgrade yfinance

In [48]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to detect Signal (Bullish and Bearish)
def signal_trade(df, shift_1= 1, shift_2= 2, shift_3= 3, shift_4=4):
    """
    Detect bullish and bearish fractals based on local min/max.
    """
    df['Bullish_Signal'] = (df['Low'] < df['Low'].shift(shift_1)) & (df['Low'].shift(shift_1) < df['Low'].shift(shift_2)) & (df['Low'].shift(shift_2) < df['Low'].shift(shift_3)) & (df['Low'].shift(shift_3) < df['Low'].shift(shift_4))
    df['Bearish_Signal'] = (df['High'] > df['High'].shift(shift_1)) & (df['High'].shift(shift_1) > df['High'].shift(shift_2)) & (df['High'].shift(shift_2) > df['High'].shift(shift_3)) & (df['High'].shift(shift_3) > df['High'].shift(shift_4))
    return df

# Function to calculate Squeeze (Bollinger Bands inside Keltner Channels)
def calculate_squeeze(df, bollinger_window=20, keltner_window=20, num_std=2):
    """
    Calculate TTM Squeeze condition where Bollinger Bands are inside Keltner Channels.
    """
    # Calculate Bollinger Bands
    bb_center = df['Close'].rolling(window=bollinger_window).mean()
    bb_std = df['Close'].rolling(window=bollinger_window).std()
    bb_upper = bb_center + (bb_std * num_std)
    bb_lower = bb_center - (bb_std * num_std)

    # Calculate Keltner Channels
    keltner_range = df['High'].rolling(window=keltner_window).max() - df['Low'].rolling(window=keltner_window).min()
    keltner_center = df['Close'].rolling(window=keltner_window).mean()
    keltner_upper = keltner_center + (keltner_range * num_std)
    keltner_lower = keltner_center - (keltner_range * num_std)

    # Squeeze condition: Bollinger Bands inside Keltner Channels
    squeeze = (bb_upper < keltner_upper) & (bb_lower > keltner_lower)

    return squeeze

# Define the stock symbol and time period
symbol = 'META'  # SPY is the symbol for the S&P 500 ETF
start_date = '2019-01-01'
end_date = '2024-11-20'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate Fractals (Bullish and Bearish)
df = signal_trade(df)

# Calculate Squeeze
df['Squeeze'] = calculate_squeeze(df, bollinger_window=20, keltner_window=20, num_std=2)

# Define Entry and Exit signals based on Fractals and Squeeze
df['Entry'] = (
    (df['Bullish_Signal']) &
    (df['Squeeze']) &
    (df['Close'] > df['Close'].shift(1))  # Additional confirmation: Price above the previous candle's high
)

df['Exit'] = (
    (df['Bearish_Signal']) &
    (df['Squeeze']) &
    (df['Close'] < df['Close'].shift(1))  # Additional confirmation: Price below the previous candle's low
)

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-11-19 00:00:00
Period                                       1230
Start Value                              100000.0
End Value                           153540.695971
Total Return [%]                        53.540696
Benchmark Return [%]                   168.271263
Max Gross Exposure [%]                      100.0
Total Fees Paid                       5360.151053
Max Drawdown [%]                        33.077608
Max Drawdown Duration                       661.0
Total Trades                                   21
Total Closed Trades                            20
Total Open Trades                               1
Open Trade PnL                        1678.995891
Win Rate [%]                                 70.0
Best Trade [%]                          22.126212
Worst Trade [%]                        -24.142602
Avg Winning Trade [%]                     7.48083
Avg Losing Trade [%]                    -8.759994
